In [1]:
# 隐藏输出
%%capture
# 如果你之前没有安装过 Unsloth，进行基础安装。
!pip install unsloth
# 目的是卸载已有的 unsloth 版本，确保不会有冲突，然后从 GitHub 安装最新的版本。
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

1. ### QLoRA（Quantized LoRA）：是 LoRA 的一种扩展，它通过使用量化技术（quantization）来减少模型的计算需求和内存使用。在 QLoRA 中，模型的权重被量化为 4 位（bit），从而极大地减少了存储和计算成本。这对显存有限的设备特别有利，如 GPU 内存较小的情况下运行大模型。
2. ### 2倍加速：Unsloth 框架通过优化底层计算和数据加载流程，特别是利用量化后的权重，在不牺牲太多模型性能的情况下，将微调和推理速度提升至常规速度的 2倍。
3. ### max_seq_length（最大序列长度）：指的是模型可以处理的输入文本的最大长度，通常以字词或 token 为单位。
4. ### RoPE Scaling（旋转位置编码扩展）：Unsloth 采用了由 Kaiokendev 开发的一种扩展 RoPE 的方法，使得用户可以随意设置 max_seq_length，而无需担心模型在较长序列上性能下降。

In [2]:
# 引入库和设置参数
import os
from unsloth import FastLanguageModel
import torch

# 定义模型的最大序列长度和精度
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# 启用 4-bit 量化
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 预先量化的 4-bit 模型列表
# fourbit_models = [
#     "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
#     "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
#     "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
#     "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
#     "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/gemma-2-9b-bnb-4bit",
#     "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
# ] # More models at https://huggingface.co/unsloth

# 加载模型
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.10.6: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla P40. Max memory: 22.376 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 6.1. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


### We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    # LoRA的秩，选择大于0的任何数字！建议使用8、16、32、64、128，较大的秩值可能会提高模型的表达能力，但也会增加计算资源的需求。
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    # 指定将应用LoRA适配器的模块
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    # LoRA的缩放因子，用于控制适配器对模型的影响。较大的值意味着适配器的影响更大。
    lora_alpha = 16,
    # LoRA适配器的dropout率，设置为0可以优化性能
    lora_dropout = 0, # Supports any, but = 0 is optimized
    # 控制是否添加偏置，设置为"none"避免在 LoRA 层中添加不必要的偏置。
    bias = "none",    # Supports any, but = "none" is optimized
    # 启用梯度检查点，可以减少内存使用
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    # 随机种子，保证结果可复现
    random_state = 3407,
    # 是否使用秩稳定的LoRA
    use_rslora = False,  # We support rank stabilized LoRA
    # LoftQ配置，适用于某些特定的量化方法
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [4]:
# 定义一个包含指令、输入和输出的模板
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# 获取 EOS_TOKEN，确保生成文本以结束符号结尾
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

# 定义格式化函数，将数据集中的每个样本转换为指定格式
def formatting_prompts_func(examples):
    # 提取指令、输入和输出
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    # 初始化一个空列表，用于存储格式化后的文本
    texts = []
    # 遍历每个样本，将其格式化为预定义的模板
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        # 将指令、输入和输出填充到模板中，并添加 EOS_TOKEN
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        # 将格式化后的文本添加到列表中
        texts.append(text)
    # 返回一个包含所有文本的字典
    return { "text" : texts, }
pass

# 从 datasets 库加载 Alpaca 数据集，指定使用训练集
from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
# 将格式化函数应用于数据集，以批量处理样本
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md: 0.00B [00:00, ?B/s]

alpaca_data_cleaned.json: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import time

# 创建 SFTTrainer 实例
trainer = SFTTrainer(
    model=model,  # 被训练的模型
    tokenizer=tokenizer,  # 对应的分词器
    train_dataset=dataset,  # 训练数据集
    dataset_text_field="text",  # 数据集中使用的文本字段
    max_seq_length=max_seq_length,  # 最大序列长度
    dataset_num_proc=2,  # 数据处理时的并行进程数
    packing=False,  # 短序列训练时是否启用打包以加快速度
    args=TrainingArguments(  # 训练参数配置
        per_device_train_batch_size=2,  # 每个设备的训练批量大小
        gradient_accumulation_steps=4,  # 梯度累积步骤
        warmup_steps=5,  # 预热步骤
        max_steps=60,  # 最大训练步骤
        learning_rate=2e-4,  # 学习率
        fp16=not is_bfloat16_supported(),  # 使用 FP16 精度
        bf16=is_bfloat16_supported(),  # 使用 BF16 精度（如果支持）
        logging_steps=1,  # 日志记录的步骤频率
        optim="adamw_8bit",  # 使用的优化器
        weight_decay=0.01,  # 权重衰减
        lr_scheduler_type="linear",  # 学习率调度类型
        seed=3407,  # 随机种子
        output_dir="outputs",  # 输出目录
        report_to="none",  # 报告到 WandB 等工具
    ),
)

Map (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


### Show current memory stats

In [7]:
#@title Show current memory stats
# 获取当前设备（GPU 0）的属性
gpu_stats = torch.cuda.get_device_properties(0)

# 获取当前训练过程已保留的最大 GPU 内存，并转换为 GB
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)

# 获取 GPU 的总内存，并转换为 GB
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

# 打印 GPU 名称和总内存
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")

# 打印当前已保留的 GPU 内存
print(f"{start_gpu_memory} GB of memory reserved.")


GPU = Tesla P40. Max memory = 22.376 GB.
11.654 GB of memory reserved.


In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers, TRL and Unsloth!
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


Step,Training Loss
1,0.537200
2,0.416300
3,0.590300
4,0.542800
5,0.629000
6,0.578700
7,0.453400
8,0.657100
9,0.540200
10,0.722300


### Show final memory and time stats

In [13]:
#@title Show final memory and time stats

# 计算并转换当前已保留的 GPU 内存为 GB
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)

# 计算 LoRA 适配器使用的内存
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)

# 计算已使用内存占总内存的百分比
used_percentage = round(used_memory / max_memory * 100, 3)

# 计算用于 LoRA 适配器的内存占总内存的百分比
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

# 打印训练所用的总时间（秒）
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")

# 打印训练所用的总时间（分钟）
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")

# 打印训练期间的峰值保留内存
print(f"Peak reserved memory = {used_memory} GB.")

# 打印用于训练的峰值保留内存
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")

# 打印保留内存占最大内存的百分比
print(f"Peak reserved memory % of max memory = {used_percentage} %.")

# 打印用于训练的内存占最大内存的百分比
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")


1542.2777 seconds used for training.
25.7 minutes used for training.
Peak reserved memory = 11.654 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 52.083 %.
Peak reserved memory for training % of max memory = 0.0 %.


### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [14]:
# alpaca_prompt = Copied from above
# 启用模型进行推理，以实现更快的推理速度（2倍）
FastLanguageModel.for_inference(model)

# 使用分词器将输入数据编码为张量
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Continue the fibonnaci sequence.", # 指令
            "1, 1, 2, 3, 5, 8", # 输入序列
            "", # 输出 - 留空以进行生成!
        )
    ], 
    return_tensors="pt" # 返回 PyTorch 格式的张量
).to("cuda") # 将数据移动到 GPU

# 生成输出，最大新生成的 token 数为 64，使用缓存
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

# 解码生成的输出，并将其转换为可读文本
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n13, 21, 34, 55, 89, 144, 233, 377, 610, 987<|end_of_text|>']

You can also use a TextStreamer for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [15]:
# alpaca_prompt = Copied from above
# 启用模型进行推理，以实现更快的推理速度（2倍）
FastLanguageModel.for_inference(model)

# 使用分词器将输入数据编码为张量
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Continue the fibonnaci sequence.", # 指令
            "1, 1, 2, 3, 5, 8", # 输入序列
            "", # 输出 - 留空以进行生成!
        )
    ], 
    return_tensors="pt" # 返回 PyTorch 格式的张量
).to("cuda") # 将数据移动到 GPU

# 从 transformers 导入 TextStreamer 以实现逐步生成
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer) # 创建 TextStreamer 实例

# 使用 streamer 进行生成，以逐个 token 显示生成过程
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 629145


### Saving, loading finetuned models

In [16]:
# 将训练好的模型保存到本地目录 "lora_model"
model.save_pretrained("lora_model") 

# 将与模型相关的分词器保存到本地目录 "lora_model"
tokenizer.save_pretrained("lora_model")

# 将模型推送到 Hugging Face Hub（在线保存），需要提供用户名和访问令牌
# model.push_to_hub("your_name/lora_model", token="...")

# 将分词器推送到 Hugging Face Hub（在线保存），需要提供用户名和访问令牌
# tokenizer.push_to_hub("your_name/lora_model", token="...")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set False to True:

In [17]:
# 如果条件为 False，这部分代码不会被执行
if False:
    # 从 unsloth 导入 FastLanguageModel
    from unsloth import FastLanguageModel
    
    # 从指定的目录加载训练好的模型和分词器
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="lora_model",  # 使用的训练模型名称
        max_seq_length=max_seq_length,  # 设置最大序列长度
        dtype=dtype,  # 设置数据类型
        load_in_4bit=load_in_4bit,  # 是否以 4 位量化加载模型
    )
    
    # 启用推理模式，以实现 2 倍更快的推理速度
    FastLanguageModel.for_inference(model)

# alpaca_prompt = 你必须从上面复制这一部分内容！

# 使用分词器将输入数据编码为张量
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "What is a famous tall tower in Paris?",  # 指令
            "",  # 输入 - 留空以进行生成!
            "",  # 输出 - 留空以进行生成!
        )
    ], 
    return_tensors="pt"  # 返回 PyTorch 格式的张量
).to("cuda")  # 将数据移动到 GPU

# 从 transformers 导入 TextStreamer 以实现逐步生成
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)  # 创建 TextStreamer 实例

# 使用 streamer 进行生成，以逐个 token 显示生成过程
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is a famous tall tower in Paris?

### Input:


### Response:
One of the most famous and iconic tall towers in Paris is the Eiffel Tower. Standing at a height of 324 meters (1,063 feet), it was built in 1889 for the World's Fair and was designed by the French engineer Gustave Eiffel. The tower is a symbol of Paris and is visited by millions of people each year.<|end_of_text|>


### GGUF / llama.cpp Conversion

In [18]:
# 保存为 8bit Q8_0 格式
if False: 
    model.save_pretrained_gguf("model", tokenizer)

# 提醒用户访问 Hugging Face 设置页面以获取令牌，并将 hf 替换为用户名
# And change hf to your username!
if False: 
    model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# 保存为 16bit GGUF 格式
if False: 
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")

if False: 
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# 保存为 q4_k_m GGUF 格式
if False: 
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

if False: 
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# 使用多个 GGUF 选项进行保存 - 如果你想同时保存多个，这会更快
if False:
    model.push_to_hub_gguf(
        "hf/model", # 将 hf 替换为你的用户名!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m"],
        token = "",
    )
